# Imports

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [85]:
block_size = 256
batch_size = 8

# Dataloader

In [96]:
chars = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',' ', '+', '-', '.', '=']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 15


In [97]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [115]:
def get_batch():
    numbers = torch.randn(batch_size, block_size, 2)
    #probs = [["{} + {} = {}".format(numbers[batch_no, ix, 0], numbers[batch_no, ix,1], numbers[batch_no, ix,0] + numbers[batch_no, ix,1]) for ix in range(block_size)] for batch_no in range(batch_size)]
    x = [["{} + {}".format(numbers[batch_no, ix, 0], numbers[batch_no, ix,1]) for ix in range(block_size)] for batch_no in range(batch_size)]
    y = [["{}".format(numbers[batch_no, ix,0] + numbers[batch_no, ix,1]) for ix in range(block_size)] for batch_no in range(batch_size)]
    return x, y

In [116]:
x = get_batch()

In [123]:
len(x[0][0])

256

In [117]:
len(x[0])

8

In [118]:
len(x)

2

In [81]:
encode(x)

KeyError: '1.3316987752914429 + 1.155342936515808'